# Tank detection YoloV8 model train

This notebook will train a [Yolov8](https://github.com/ultralytics/ultralytics) model for tank detection using publicly available annotated images of tanks.

As the notebook will run the training with `PyTorch`, it is recommended to have GPUs available. If running in Google Colab, go to Edit > Notebook settings and select GPU hardware acceleration.

The first step of the notebook will use [fiftyone](https://github.com/voxel51/fiftyone), to build a dataset of tank images and bounding box annotations for training the computer vision model. The tutorial notebook on [Fine-tuning YOLOv8 models for custom use cases](https://github.com/voxel51/fiftyone/blob/v0.21.0/docs/source/tutorials/yolov8.ipynb) is a usefull introduction on how to use `fiftyone`.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

To start, check GPU support.

In [2]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.3.0+cu121 (NVIDIA RTX A6000)


We also setup logs.

In [3]:
import logging

logging.basicConfig(level=logging.INFO)

## Download images from ImageNet

The first dataset we'll use is ImageNet21k. The ImageNet21k dataset is available at [https://image-net.org/download-images.php](https://image-net.org/download-images.php). You need to register and be granted access to download the images. We use the Winter 21 version since it gives the option of downloading the images for a single synset: https://image-net.org/data/winter21_whole/SYNSET_ID.tar, e.g., https://image-net.org/data/winter21_whole/n02352591.tar. The processed version of ImageNet21k is available here : https://github.com/Alibaba-MIIL/ImageNet21K. The class ids and names are available here https://github.com/google-research/big_transfer/issues/7#issuecomment-640048775.

We'll begin by downloading the class names that are in ImageNet21k and look for relevant classes that we can use.

In [4]:
from pathlib import Path

imagenet_dir = Path() / "imagenet"

In [5]:
from adomvi.datasets.imagenet import download_class_names, find_class_by_text

classes = download_class_names(imagenet_dir)
find_class_by_text(classes, "military")

INFO:root:File imagenet/imagenet21k_wordnet_ids.txt already exists. Skipping download.
INFO:root:File imagenet/imagenet21k_wordnet_lemmas.txt already exists. Skipping download.


{'n03762982': 'military_hospital',
 'n03763727': 'military_quarters',
 'n03763968': 'military_uniform',
 'n03764276': 'military_vehicle',
 'n04552348': 'warplane, military_plane',
 'n08249459': 'concert_band, military_band',
 'n09809538': 'army_engineer, military_engineer',
 'n09943239': 'commissioned_military_officer',
 'n10316360': 'military_attache',
 'n10316527': 'military_chaplain, padre, Holy_Joe, sky_pilot',
 'n10316862': 'military_leader',
 'n10317007': 'military_officer, officer',
 'n10317500': 'military_policeman, MP',
 'n10512372': 'recruit, military_recruit',
 'n10582746': 'serviceman, military_man, man, military_personnel',
 'n10759331': 'volunteer, military_volunteer, voluntary'}

We can now download images and annotations for the relevant classes. The `download_imagenet_detections` function will download the images and annotations for the given class ids **if the annotations exist** (not all classes have been annotated).

In [6]:
from adomvi.datasets.imagenet import download_imagenet_detections

class_ids = ["n02740300", "n04389033", "n02740533", "n04464852", "n03764276"]
download_imagenet_detections(class_ids, imagenet_dir)

INFO:root:File imagenet/bboxes_annotations.tar.gz already exists. Skipping download.
INFO:root:There are not annotations for class n02740300.
INFO:root:Annotations directory imagenet/labels/n04389033 already exists. Skipping extract.
INFO:root:There are not annotations for class n02740533.
INFO:root:There are not annotations for class n04464852.
INFO:root:There are not annotations for class n03764276.
INFO:root:Deleting annotations dir.


The data we just downloaded into the `imagenet` directory is not all clean: there are annotations which have no corresponding image. We need to remove those labels, otherwise this causes errors when importing the data into fiftyone.

In [7]:
from adomvi.datasets.imagenet import cleanup_labels_without_images

cleanup_labels_without_images(imagenet_dir)

INFO:root:Deleting 0 labels without images


We can now create a new dataset with `fiftyone`. Fiftyone allows us to manage images annotated with bounding boxes and labels, to merge datasets from different sources, and to split the datasets and prepare them for processing.

In [8]:
import fiftyone as fo

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=imagenet_dir,
    dataset_type=fo.types.VOCDetectionDataset,
)

dataset.name = "military-vehicles"

dataset.map_labels(
    "ground_truth",
    {"n04389033":"tank"}
).save()

 100% |█████████████████| 378/378 [458.6ms elapsed, 0s remaining, 827.5 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 378/378 [458.6ms elapsed, 0s remaining, 827.5 samples/s]      


Once our dataset is created, we can launch a session to display the dataset and view the annotated images

In [9]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [10]:
session.open_tab()

<IPython.core.display.Javascript object>

### Add OpenImage samples

The ImageNet dataset only contained 378 annotated images of tanks, so we'll look into other available datasets to improve training of the model. We’ll load [Open Images](https://storage.googleapis.com/openimages/web/index.html) samples with `Tank` detection labels, passing in `only_matching=True` to only load the `Tank` labels. We then map these labels by changing `Tank` into `tank`.

In [11]:
import fiftyone.zoo as foz

oi_samples = foz.load_zoo_dataset(
    "open-images-v7",
    classes = ["Tank"],
    only_matching=True,
    label_types="detections"
).map_labels(
    "ground_truth",
    {"Tank":"tank"}
)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/home/jrenault/fiftyone/open-images-v7/train' if necessary


Necessary images already downloaded


INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'train' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'train' is sufficient


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'validation'


 100% |███████████████████| 50/50 [85.3ms elapsed, 0s remaining, 586.3 samples/s]     


INFO:eta.core.utils: 100% |███████████████████| 50/50 [85.3ms elapsed, 0s remaining, 586.3 samples/s]     


Dataset 'open-images-v7' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-v7' created


In [12]:
dataset.merge_samples(oi_samples)

## Fine-tune a YOLOv8 detection model

Now that our dataset is created, we'll export it into a format supported by YOLOv8 to train our model.

We first remove tags from the dataset, and split it into a train, val and test sets.

In [13]:
import fiftyone.utils.random as four

## delete existing tags to start fresh
dataset.untag_samples(dataset.distinct("tags"))

## split into train, test and val
four.random_split(dataset, {"train": 0.8, "val": 0.1, "test": 0.1})

Once our dataset is split, we can export it to a specific directory.

In [17]:
from pathlib import Path
from adomvi.yolo.utils import export_yolo_data

export_dir = Path() / "vehicles"
export_yolo_data(dataset, export_dir, ["tank"], split = ["train", "val", "test"], overwrite=True)

 100% |███████████████| 1299/1299 [1.3s elapsed, 0s remaining, 979.9 samples/s]        


INFO:eta.core.utils: 100% |███████████████| 1299/1299 [1.3s elapsed, 0s remaining, 979.9 samples/s]        


Directory 'vehicles' already exists; export will be merged with existing files


 100% |█████████████████| 163/163 [166.7ms elapsed, 0s remaining, 977.8 samples/s]     


INFO:eta.core.utils: 100% |█████████████████| 163/163 [166.7ms elapsed, 0s remaining, 977.8 samples/s]     


Directory 'vehicles' already exists; export will be merged with existing files


 100% |█████████████████| 162/162 [189.8ms elapsed, 0s remaining, 853.4 samples/s]     


INFO:eta.core.utils: 100% |█████████████████| 162/162 [189.8ms elapsed, 0s remaining, 853.4 samples/s]     


Finally, we can train a YoloV8 base model on our exported dataset. We use the `yolov8n.pt` (nano) model, which is the smallest, but larger models are available from ultralytics. Simply replace `yolov8n.pt` with the [right model](https://docs.ultralytics.com/models/yolov8/#key-features).

In [19]:
from adomvi.yolo.yolo import train

train("yolov8n.pt", data=export_dir / "dataset.yaml", batch=64, device=[0, 1])

Ultralytics YOLOv8.2.28 🚀 Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 48655MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=vehicles/dataset.yaml, epochs=60, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

/home/jrenault/mambaforge/envs/adomvi2/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /home/jrenault/workspace/adomvi2/notebooks/vehicles/labels/train.cache... 1299 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1299/1299 [00:00<?, ?it/s]
val: Scanning /home/jrenault/workspace/adomvi2/notebooks/vehicles/labels/val.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|██████████| 163/163 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 16 dataloader workers
Logging results to runs/detect/train
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        163        245      0.859      0.788      0.844      0.615

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      60/60       4.3G     0.4322     0.3308     0.9603         15        640: 100%|██████████| 21/21 [00:03<00:00,  5.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.90it/s]


                   all        163        245       0.91      0.751      0.844      0.613

60 epochs completed in 0.081 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.28 🚀 Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 48655MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


/home/jrenault/mambaforge/envs/adomvi2/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.71it/s]


                   all        163        245      0.902      0.767       0.85      0.618
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train


## Use the fine-tuned model to predict detections

Once our model is trained, we can simply use it to predict detections on our test dataset. Simply pass the path to the best model weights and the directory containing the test images to the `predict` function.

In [20]:
from pathlib import Path
from adomvi.yolo.yolo import predict

best_model = Path() / "runs/detect/train/weights/best.pt"
results = predict(best_model, source=export_dir / "images/test")


image 1/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/0080cbc853059db1.jpg: 448x640 1 tank, 86.1ms
image 2/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/00c5be8db0e25f5e.jpg: 448x640 1 tank, 4.8ms
image 3/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/01368575b56715e7.jpg: 640x640 1 tank, 5.3ms
image 4/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/014df43a4a199fe0.jpg: 480x640 12 tanks, 88.6ms
image 5/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/016727715ecbb0ea.jpg: 384x640 1 tank, 92.6ms
image 6/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/021bb7e1cbb12d26.jpg: 384x640 1 tank, 8.2ms
image 7/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/025457c66e740c30.jpg: 448x640 1 tank, 7.4ms
image 8/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/02766a4a2a22632b.jpg: 640x448 2 tanks, 105.9ms
image 9/162 /home/jrenault/work

## Evaluate model predictions

We can first evaluate our model by loading the predictions in a Fiftyone dataset view of our test data. We select the images tagged as part of the test set in our dataset.

In [21]:
# The test split of the dataset
test_view = dataset.match_tags("test")

Then we can load the model predictions as fields of the images in our test view.

In [35]:
from pathlib import Path
from adomvi.yolo.utils import add_yolo_detections

prediction_field = "yolov8"
predictions_dir = Path() / "runs/detect/predict/labels"
add_yolo_detections(test_view, prediction_field=prediction_field, predictions_dir=predictions_dir, class_list=["tank"])

Now we can visualize these YOLOv8 model predictions on the samples in our dataset in the FiftyOne app:

In [36]:
session.open_tab()

<IPython.core.display.Javascript object>

Finally, we can evaluate our model's predictions and print the mAP.

In [37]:
detection_results = test_view.evaluate_detections(
    prediction_field, 
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 162/162 [569.9ms elapsed, 0s remaining, 284.3 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 162/162 [569.9ms elapsed, 0s remaining, 284.3 samples/s]      


Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 162/162 [585.3ms elapsed, 0s remaining, 276.8 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 162/162 [585.3ms elapsed, 0s remaining, 276.8 samples/s]      


In [38]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

mAP = 0.5826986060627386


In [39]:
detection_results.print_report()

              precision    recall  f1-score   support

        tank       0.85      0.79      0.82       260

   micro avg       0.85      0.79      0.82       260
   macro avg       0.85      0.79      0.82       260
weighted avg       0.85      0.79      0.82       260

